In [ ]:
%cd ..

In [ ]:
import os

import pandas as pd
import wikipediaapi as wk
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from tqdm import tqdm

from prompts import load_template
from utils import create_experiment_file_name

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
WIKI_USER_AGENT = os.getenv("WIKI_USER_AGENT")

DATA_DIR = "data"
RESULT_DIR = "experiments"
TARGET_DATA = "train_v2.0.1.csv"
EXP_VERSION = "v1.0.2"  # 컨벤션에 따라 새롭게 실험할 경우 버저닝 정보를 기록해주세요.

PARAGRAPH = "paragraph"
QUESTION = "question"
CHOICES = "choices"
QUESTION_PLUS = "question_plus"

In [ ]:
datasets = pd.read_csv(os.path.join(DATA_DIR, TARGET_DATA))
datasets.head()

In [ ]:
prompt = PromptTemplate.from_template(
    template=load_template(file_name="wikipedia_search_keyword.txt", template_type="keyword_extraction")
)

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini", temperature=0, max_tokens=None, timeout=None, max_retries=2, api_key=OPENAI_API_KEY
)

keyword_extractor = prompt | llm

In [ ]:
def extract_keyword(data: pd.Series) -> list[str]:
    """
    문제 해결에 필요한 키워드 5개를 추출하고 결과를 배열로 반환하는 함수

    Args:
        data (pd.Series): 제공된 데이터셋의 한 행(row), 문제 하나

    Returns:
        list[str]: 추출된 5개 키워드
    """
    response = keyword_extractor.invoke(
        {
            "paragraph": data[PARAGRAPH],
            "question": data[QUESTION],
            "question_plus": data[QUESTION_PLUS],
            "choices": data[CHOICES],
        },
    )
    response_split = response.content.split(",")
    keywords = [keyword.strip() for keyword in response_split]
    return keywords


tqdm.pandas()
datasets["keywords"] = datasets.progress_apply(extract_keyword, axis=1)
datasets.head()

In [ ]:
result_file_name = create_experiment_file_name(target_file=TARGET_DATA, version=EXP_VERSION)
result_file_path = os.path.join(DATA_DIR, RESULT_DIR, result_file_name)
datasets.to_csv(result_file_path, index=False)  # 1차 중간 결과

추출된 키워드가 위키피디아에 존재하는지 여부 확인

- 키워드 추출 결과가 유효한지 확인하기 위한 작업입니다.


In [ ]:
wiki = wk.Wikipedia(user_agent=WIKI_USER_AGENT, language="ko")

In [ ]:
# 키워드를 유용하게 다루기 위해 키워드 별로 칼럼으로 분리
keywords_expanded = pd.DataFrame(datasets["keywords"].tolist(), index=datasets.index)
keywords_expanded.columns = [f"keyword_{i+1}" for i in range(keywords_expanded.shape[1])]
new_datasets = pd.concat([datasets.drop(columns=["keywords"]), keywords_expanded], axis=1)

new_datasets.head()

In [ ]:
# 위키피디아 페이지가 있는지 키워드 별로 확인
for col in keywords_expanded.columns:
    new_datasets[f"{col}_exists"] = new_datasets[col].apply(lambda x: wiki.page(x).exists())

new_datasets.head()

In [ ]:
# _exists 열만 추출
exists_columns = [col for col in new_datasets.columns if col.endswith("_exists")]

true_count = new_datasets[exists_columns].sum().sum()  # True 값의 총합
false_count = (new_datasets[exists_columns] == False).sum().sum()  # False 값의 총합

true_count, false_count

In [ ]:
new_datasets.to_csv(result_file_path)